# 02 - Preprocesamiento y Feature Engineering

**Objetivo:** En este notebook, se prepara el dataset para el entrenamiento de modelos. Se aplican las decisiones tomadas en el EDA, se seleccionan las características más importantes, se dividen los datos y se escalan para dejarlos en un formato óptimo para los algoritmos de Machine Learning.

## 1. Carga de Datos e Importaciones
Importamos las librerías necesarias y cargamos el dataset limpio y optimizado que generamos en el primer notebook.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from joblib import dump

In [2]:
# Cargar el dataset optimizado
df = pd.read_parquet(r'../data/processed/cic_ids_2017_optimized.parquet')

## 2. Selección Manual de Características
Basado en la alta correlación (0.89) encontrada en el EDA entre `Flow Duration` y `Flow IAT Mean`, eliminamos esta última para reducir la redundancia en el modelo.

In [3]:
# Selección Manual (Basada en Correlación)
# En el EDA, vimos una correlación muy alta (0.89) entre 'Flow Duration' y 'Flow IAT Mean'.
# Decidimos eliminar 'Flow IAT Mean' para reducir la redundancia.
df.drop(columns = ['Flow IAT Mean'], inplace = True)

## 3. División en Características (X) y Objetivo (y)
Separamos nuestro DataFrame en dos: `X` contendrá todas las características predictoras y `y` contendrá la columna objetivo que queremos predecir (`Label`).

In [4]:
X = df.drop('Label', axis = 1)
y = df['Label']

## 4. División en Conjuntos de Entrenamiento y Prueba (Train/Test Split)
Dividimos los datos en un conjunto de entrenamiento (70%) y uno de prueba (30%). Es crucial hacer esto **antes** de aplicar técnicas como la selección automática o el escalado para evitar la fuga de datos (data leakage).

Se utiliza `stratify = y` para asegurar que la proporción de clases (el desbalance) sea la misma tanto en el conjunto de entrenamiento como en el de prueba.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

## 5. Selección Automática de Características
Utilizamos un modelo `RandomForestClassifier` como "juez" a través de `SelectFromModel` para evaluar la importancia de las características restantes. Nos quedamos con aquellas cuya importancia está por encima de la mediana, reduciendo así la dimensionalidad y enfocándonos en las variables más informativas.

In [6]:
# Creamos el modelo que actuará como selector
selector_model = RandomForestClassifier(n_estimators = 50, random_state = 42, n_jobs= -1)

# Creamos el objeto selector, que elegirá las características con importancia > a la mediana
selector = SelectFromModel(estimator = selector_model, threshold = 'median')

# Entrenamos el selector
selector.fit(X_train, y_train)

# Obtenemos los nombres de las columnas seleccionadas
selected_features_mask = selector.get_support()
selected_features_names = X_train.columns[selected_features_mask]

# Transformamos nuestros conjuntos para quedarnos solo con las columnas seleccionadas
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Reportamos los resultados
print("\nProceso de selección completado.")
print("Número original de características:", X_train.shape[1])
print("Número de características seleccionadas:", X_train_selected.shape[1])
print("\nCaracterísticas seleccionadas:")
print(selected_features_names.tolist())

# Convertimos los arrays de numpy de vuelta a DataFrames de Pandas
X_train_selected = pd.DataFrame(X_train_selected, columns = selected_features_names)
X_test_selected = pd.DataFrame(X_test_selected, columns = selected_features_names)


Proceso de selección completado.
Número original de características: 77
Número de características seleccionadas: 39

Características seleccionadas:
['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'PSH Flag Count', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length.1', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward']

## 6. Escalado de Características Numéricas
Aplicamos un `StandardScaler` para estandarizar las características numéricas (media 0, desviación estándar 1). El escalador se "ajusta" (`.fit()`) **únicamente** con los datos de entrenamiento y luego se aplica (`.transform()`) a ambos conjuntos (entrenamiento y prueba) para mantener la consistencia.

In [7]:
# Creamos el escalador
scaler = StandardScaler()

# "Ajustamos" el escalador SÓLO con los datos de entrenamiento para que aprenda la media y desviación estándar
scaler.fit(X_train_selected)

# "Transformamos" tanto el conjunto de entrenamiento como el de prueba
X_train_scaled = scaler.transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

## 7. Guardado de los Datos Preprocesados

Finalmente, guardamos todos los conjuntos de datos (`X_train_scaled`, `X_test_scaled`, `y_train`, `y_test`) y la lista de características seleccionadas en un único archivo `.joblib`.

Esto nos permitirá cargar estos datos ya listos y procesados directamente en el siguiente notebook (`03_model_training`), sin necesidad de volver a ejecutar todo este pipeline de preprocesamiento.

In [8]:
# Creamos un diccionario para guardar todos nuestros datos de forma ordenada
datos_preprocesados = {
    'X_train_scaled': X_train_scaled,
    'X_test_scaled': X_test_scaled,
    'y_train': y_train,
    'y_test': y_test,
    'selected_features': selected_features_names # Guardamos también los nombres de las columnas
}

# Definimos la ruta de guardado
ruta_guardado = r'../data/processed/cicids2017_preprocessed_data.joblib'

# Guardamos el diccionario en un único archivo
dump(datos_preprocesados, ruta_guardado)

['../data/processed/cicids2017_preprocessed_data.joblib']